This notebook will let the user see how the results of machine learning models will change when a different number of features are used. 

Lets load the API.

In [1]:
# import the API
APILoc = r"C:\Users\chris\Documents\Thesis\code\scripts\API"

import sys
sys.path.insert(0, APILoc)

from API import *

This cell will allow the user to view all of the results of saved machine learning models from a given directory.

In [ ]:
# get the location for the models

# directory
allFeatModelsLoc = r"*insert the directory where all of the results are saved*"

# list of names of all of the files in the above directory whose content should be printed out
modelList = ["dtTup.pkl", "rfTup.pkl", "knnTup.pkl", "svrTup.pkl", "nnTup.pkl", "bayesTup.pkl", "linRegTup.pkl"]

# for each file, print out the results
for model in modelList:
    # load the tuples
    location = allFeatModelsLoc + "\\" + model
    aTuple = joblib.load(location) # has the form: (bestModel, bestFeatures, bestMAE, bestRSq, bestR, avgMAE, 
                                   #                avgRSq, avgR, bestMSE, bestMAPE, avgMSE, avgMAPE, trainedModelsList)
    
    # get model name
    stopIndex = model.find(".")
    modelName = model[:stopIndex]
    
    # get values
    
    hyperParams = aTuple[0].best_params_
    bestFeatures = aTuple[1]
    bestMAE = aTuple[2]
    bestRSq = aTuple[3]
    bestR = aTuple[4]
    avgMAE = aTuple[5]
    avgRSq = aTuple[6]
    avgR = aTuple[7]
    bestMSE = aTuple[8]
    bestMAPE = aTuple[9]
    avgMSE =aTuple[10]
    avgMAPE = aTuple[11]
    
    # print results
    print(modelName + " avg MAE: ", round(avgMAE, 3))
    print(modelName + " avg MSE: ", round(avgMSE, 3))
    print(modelName + " avg MAPE: ", round(avgMAPE, 3))
    print(modelName + " avg R: ", round(avgR,3))
    print(modelName + " avg R Squared: ", avgRSq)
    print(modelName + " best MAE: ", round(bestMAE, 3))
    print(modelName + " best MSE: ", round(bestMSE, 3))
    print(modelName + " best MAPE: ", round(bestMAPE, 3))
    print(modelName + " best R: ", round(bestR, 3))
    print(modelName + " best R Squared: ", bestRSq)
    print(modelName + " parameters for best model: ", hyperParams)
    print(modelName + " features used: ", bestFeatures)
    print(" ")

The next cell will produce a graph showing how the results change with different number of features. To work, there must be a working directory which itself contains folders called 'Nfeatures', where 'N' is an integer. Each 'Nfeatures' folder, must only contain the results from the function 'saveMLResults'.

In [ ]:
# the directory which contains all of the folders listed in 'aFeaturesList'
aWorkingDir = r"*insert location of working directory*"

# list of folder names where each folder's name has the format 'Nfeatures'
aFeaturesList = ["3features", "4features", "5features", "6features", "7features", "8features", "9features",
                "10features", "11features"]

# the name of the files in each of the folders named in 'aFeaturesList'
aModelList = ["rfTup.pkl", "knnTup.pkl", "svrTup.pkl","nnTup.pkl","dtTup.pkl"]

plotFeaturesAndR(aWorkingDir, aFeaturesList, aModelList)